# Sprint 13 - Proyecto

### Libraries loading (Cargar las librerías)

In [14]:
# Data manipulation
from pathlib import Path
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt

# Machine learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# Metrics
from sklearn.metrics import (
    roc_auc_score,
    roc_curve,
    auc,
    f1_score,
    mean_squared_error
)

# Utilities
from sklearn.utils import shuffle


### Data loading (cargar los datos)

In [16]:
#Crea un objeto Path que representa el directorio actual, .resolve() convierte esa ruta en una ruta absoluta completa
BASE_DIR = Path().resolve()
DATA_DIR = BASE_DIR / "data" 

#Define las rutas de los archivos
train_path = DATA_DIR / "gold_recovery_train.csv"
test_path = DATA_DIR / "gold_recovery_test.csv"
full_path = DATA_DIR / "gold_recovery_full.csv" 

# Cargar datasets
gold_recovery_train = pd.read_csv(train_path)
gold_recovery_test = pd.read_csv(test_path)
gold_recovery_full = pd.read_csv(full_path)


### Exploratory Data Analysis (Análisis exploratorio de datos)

In [24]:
datasets = {
    "Train": gold_recovery_train,
    "Test": gold_recovery_test,
    "Full": gold_recovery_full
}

#Cálculo de dimensiones de los datasets
print("Dimensiones de los datasets:\n")

for name, df in datasets.items():
    print(f"{name}: {df.shape}")

print("\nValores ausentes:\n")

#Cálculo de valores ausentes en cada dataset
for name, df in datasets.items():
    total_missing = df.isnull().sum().sum()
    total_values = df.size
    missing_percent = (total_missing / total_values) * 100
    
    print(f"{name} - Total valores ausentes: {total_missing} ({missing_percent:.2f}%)")
    
#Cálculo de valores ausentes por cada columna de cada dataset
for name, df in datasets.items():
    print(f"\nAnálisis de valores ausentes - {name}")
    
    missing_df = pd.DataFrame({
        "missing_count": df.isnull().sum(),
        "missing_percent": (df.isnull().mean() * 100).round(2)
    }).sort_values(by="missing_percent", ascending=False)
    
    print(missing_df.head(10))


Dimensiones de los datasets:

Train: (16860, 87)
Test: (5856, 53)
Full: (22716, 87)

Valores ausentes:

Train - Total valores ausentes: 30320 (2.07%)
Test - Total valores ausentes: 2360 (0.76%)
Full - Total valores ausentes: 36587 (1.85%)

Análisis de valores ausentes - Train
                                    missing_count  missing_percent
rougher.output.recovery                      2573            15.26
rougher.output.tail_ag                       2250            13.35
rougher.output.tail_au                       2249            13.34
rougher.output.tail_sol                      2249            13.34
secondary_cleaner.output.tail_sol            1986            11.78
rougher.input.floatbank11_xanthate           1904            11.29
final.output.recovery                        1521             9.02
primary_cleaner.input.sulfate                1307             7.75
primary_cleaner.input.depressant             1262             7.49
rougher.calculation.au_pb_ratio              1242    

In [20]:
# Identificar las columnas necesarias, aquellas relacionadas conrougher

rougher_cols = [col for col in gold_recovery_train.columns if 'rougher' in col]
print('Columnas rougher disponibles:')
for col in rougher_cols:
    print(f" - {col}")

Columnas rougher disponibles:
 - rougher.calculation.sulfate_to_au_concentrate
 - rougher.calculation.floatbank10_sulfate_to_au_feed
 - rougher.calculation.floatbank11_sulfate_to_au_feed
 - rougher.calculation.au_pb_ratio
 - rougher.input.feed_ag
 - rougher.input.feed_pb
 - rougher.input.feed_rate
 - rougher.input.feed_size
 - rougher.input.feed_sol
 - rougher.input.feed_au
 - rougher.input.floatbank10_sulfate
 - rougher.input.floatbank10_xanthate
 - rougher.input.floatbank11_sulfate
 - rougher.input.floatbank11_xanthate
 - rougher.output.concentrate_ag
 - rougher.output.concentrate_pb
 - rougher.output.concentrate_sol
 - rougher.output.concentrate_au
 - rougher.output.recovery
 - rougher.output.tail_ag
 - rougher.output.tail_pb
 - rougher.output.tail_sol
 - rougher.output.tail_au
 - rougher.state.floatbank10_a_air
 - rougher.state.floatbank10_a_level
 - rougher.state.floatbank10_b_air
 - rougher.state.floatbank10_b_level
 - rougher.state.floatbank10_c_air
 - rougher.state.floatbank10_

### Data Preprocessing (Preprocesamiento de datos)

In [21]:
# Realizar el cálculo de la característica rougher.output.recovery de forma manual

#C: concentración de la sustancia en el concentrado
C = gold_recovery_train['rougher.output.concentrate_au']

#F: concentración de la sustancia en la alimentación(feed)
F = gold_recovery_train['rougher.input.feed_au']

#T: concentración de la sustancia en las colas (tails)
T = gold_recovery_train['rougher.output.tail_au']

#Calcular la recuperación
recovery_calculated = (C * (F - T)/(F * (C - T))) * 100
print(f"Recovery calculada: {recovery_calculated.head().values}")

Recovery calculada: [87.10776293 86.84326051 86.84230826 87.22642953 86.68879414]


In [28]:
# Comparar los valores calculados y los valores originales

tolerance = 0.01

close_match = abs(recovery_calculated - gold_recovery_train['rougher.output.recovery']) < tolerance
print(f'Valores similares (±{tolerance}): {close_match.sum()} de {len(close_match)}')

# Calcular el MAE
mae = abs(recovery_calculated - gold_recovery_train['rougher.output.recovery']).mean()
print(f"Error Absoluto Medio: {mae}")

Valores similares (±0.01): 14287 de 16860
Error Absoluto Medio: 9.303415616264301e-15


De acuerdo con los datos calculados, se observa que 14287 datos de 16860 (aproximadamente el 84.7%) se encuentran dentro de un intervalo del ±10%, dejando algunos datos fuera por posibles causas como la forma en que se tomaron los datos, la precisión del análisis, la precisión de los instrumentos, incertidumbre de los instrumentos, error al tomar las medidas, etc.

Por otro lado, se observa que el Error Absoluto Medio es un valor muy pequeño, lo cual aporta información sobre que los datos calculados y los datos originales se encuentran muy cercanos entre ellos. 

##### 1.3. Analiza las características no disponibles en el conjunto de prueba. ¿Cuáles son estos parámetros? ¿Cuál es su tipo?

In [7]:
# Revisar qué columnas faltan en test vs train
train_cols = set(gold_recovery_train.columns)
test_cols = set(gold_recovery_test.columns)
missing_in_test = train_cols - test_cols

print(f"Columnas que están en train pero NO en test: {len(missing_in_test)}")
print("Columnas faltantes:")
print(list(missing_in_test)[:])

Columnas que están en train pero NO en test: 34
Columnas faltantes:
['final.output.tail_pb', 'secondary_cleaner.output.tail_sol', 'final.output.tail_sol', 'final.output.concentrate_au', 'rougher.output.tail_au', 'final.output.concentrate_ag', 'rougher.calculation.floatbank10_sulfate_to_au_feed', 'rougher.calculation.floatbank11_sulfate_to_au_feed', 'rougher.output.concentrate_sol', 'primary_cleaner.output.tail_sol', 'primary_cleaner.output.tail_ag', 'rougher.calculation.sulfate_to_au_concentrate', 'rougher.output.recovery', 'primary_cleaner.output.concentrate_au', 'primary_cleaner.output.concentrate_pb', 'final.output.tail_au', 'final.output.recovery', 'secondary_cleaner.output.tail_ag', 'primary_cleaner.output.tail_au', 'primary_cleaner.output.concentrate_ag', 'final.output.concentrate_sol', 'rougher.calculation.au_pb_ratio', 'rougher.output.concentrate_au', 'rougher.output.tail_ag', 'rougher.output.concentrate_ag', 'rougher.output.tail_pb', 'secondary_cleaner.output.tail_au', 'final.

Se observa que las columnas faltantes en el dataset de prueba (test) probablemente sean las variables objetivo, lo cual explicaría el porqué se encuentran en el dataset de entrenamiento pero no en el de prueba.

##### 1.4. Realiza el preprocesamiento de datos.